In [ ]:
# Install necessary libraries
!pip install torch torchvision torchmetrics thop


In [ ]:
! pip install torchprofile

In [ ]:
!pip install -U fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=767f245ae4c246f6fea291d8f79803190127f1060d770c426dae1afb62f0005d
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31532 sha256=b4cf747cef707689ce779a66a93fca07790bb344461fc29ae56d6ff4171127e3
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath


In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.transforms.functional as TF
import os


from PIL import Image

#from torchmetrics import JaccardIndex
#from thop import profile, clever_format
import time

#from fvcore.nn import FlopCountAnalysis, flop_count_table
#import torchprofile

#from models.deeplabv2.deeplabv2 import get_deeplab_v2


from torchmetrics import JaccardIndex
from thop import profile, clever_format
import time

from fvcore.nn import FlopCountAnalysis, flop_count_table
import torchprofile





**Cityscapes Dataset Download**

In [ ]:
#!pip install gdown



In [ ]:
#import gdown

#url = 'https://drive.google.com/uc?id=1Qb4UrNsjvlU-wEsR9d7rckB0YS_LXgb2'
#output = 'your_dataset.zip'  # Change this to the desired output filename
#gdown.download(url, output, quiet=False)
from google.colab import drive
import pathlib
drive.mount('/content/Drive')

get_ipython().system('/content/Drive/MyDrive/Cityspaces/Cityspaces')

Mounted at /content/Drive
/bin/bash: line 1: /content/Drive/MyDrive/Cityspaces/Cityspaces: Is a directory


In [ ]:
#get_ipython().system('unzip /content/your_dataset.zip -d /content/')

**GTA5 Dataset download**

dataset link :  https://drive.google.com/file/d/1xYxlcMR2WFCpayNrW2-Rb7N-950vvl23/view?usp=sharing

In [ ]:
#url=' https://drive.google.com/uc?id=1xYxlcMR2WFCpayNrW2-Rb7N-950vvl23'
#output = 'GTA5.zip'  # Change this to the desired output filename
#gdown.download(url, output, quiet=False)
from google.colab import drive
import pathlib
drive.mount('/content/Drive')

get_ipython().system('/content/Drive/MyDrive/GTA5')

Drive already mounted at /content/Drive; to attempt to forcibly remount, call drive.mount("/content/Drive", force_remount=True).
/bin/bash: line 1: /content/Drive/MyDrive/GTA5: Is a directory


In [ ]:
#get_ipython().system('unzip /content/GTA5.zip -d /content/')

**Model Clone**

In [ ]:
# Clone the GitHub repository
!git clone https://github.com/Gabrysse/MLDL2024_project1.git


Cloning into 'MLDL2024_project1'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 34 (delta 8), reused 4 (delta 4), pack-reused 16
Receiving objects: 100% (34/34), 12.06 KiB | 12.06 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [ ]:
# Navigate to the project directory
%cd MLDL2024_project1

/content/MLDL2024_project1


In [ ]:
#from models.deeplabv2.deeplabv2 import get_deeplab_v2
from models.bisenet.build_bisenet import BiSeNet

In [ ]:
# Set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


**Parameters**

In [ ]:
# Define training parameters
epochs = 50
learning_rate = 0.01
batch_size = 4
train_resolution = (1280,720) #(1024, 512)
test_resolution = (1024, 512)


In [ ]:

class CityscapesDataset(Dataset):
  def __init__(self, root_dir, im_transform ):

    """
    Args:
    root_dir (string): Directory with all the images.
    transform (callable, optional): Optional transform to be applied on a sample.
    """
    self.root_dir = root_dir
    self.im_transform = im_transform
    #self.lab_transform = lab_transform
    self.images = []
    for subdir, dirs, files in os.walk(root_dir):
      for file in files:
        if file.endswith('_gtFine_color.png'):
          self.images.append(os.path.join(subdir, file))

  def __len__(self):
    return len(self.images)

  def __getitem__(self, idx):
    img_name = self.images[idx]
    image = Image.open(img_name).convert('RGB')
    label_name = img_name.replace('_gtFine_color.png', '_gtFine_labelTrainIds.png')  #labelTrainIds
    label = Image.open(label_name)


    # Resize label using nearest-neighbor interpolation
    label = TF.resize(label, (1024, 512), interpolation=transforms.InterpolationMode.NEAREST)
    label = np.array(label)  # Convert to numpy array
    label = torch.from_numpy(label).long()  # Convert to LongTensor


    if self.im_transform:

      image = self.im_transform(image)

    # if self.lab_transform:
    #   label = self.lab_transform(label)

    return image, label



In [ ]:
class GTA5Dataset(Dataset):
  def __init__(self, root_dir, im_transform=None):
    """
    Args:
    root_dir (string): Directory with all the images and labels.
    im_transform (callable, optional): Optional transform to be applied on a sample.
    """
    self.root_dir = root_dir
    self.im_transform = im_transform
    self.images_dir = os.path.join(root_dir, 'images')
    self.labels_dir = os.path.join(root_dir, 'labels')
    self.images = [os.path.join(self.images_dir, file) for file in os.listdir(self.images_dir) if file.endswith('.png')]

  def __len__(self):
    return len(self.images)

  def __getitem__(self, idx):
    img_name = self.images[idx]
    image = Image.open(img_name).convert('RGB')
    label_name = img_name.replace('images', 'labels')  # Assuming label images are in the 'labels' folder with the same name
    label = Image.open(label_name)

    # Resize label using nearest-neighbor interpolation
    label = TF.resize(label, (1280,720) , interpolation=transforms.InterpolationMode.NEAREST)
    label = np.array(label)  # Convert to numpy array

    # Define the maximum valid label ID
    max_valid_id = 18
    # Set all label IDs greater than max_valid_id to the 'ignore' class (e.g., 255)
    label[label > max_valid_id] = 255


    label = torch.from_numpy(label).long()  # Convert to LongTensor

    if self.im_transform:
      image = self.im_transform(image)

    return image, label


**GTA5 Train Loader**

In [ ]:
from google.colab import drive
import pathlib
drive.mount('/content/Drive')

get_ipython().system('/content/Drive/MyDrive/GTA5')

Drive already mounted at /content/Drive; to attempt to forcibly remount, call drive.mount("/content/Drive", force_remount=True).
/bin/bash: line 1: /content/Drive/MyDrive/GTA5: Is a directory


In [ ]:
!ls /content/Drive/MyDrive/GTA5/GTA5/GTA5

ls: cannot access '/content/Drive/MyDrive/GTA5/GTA5/GTA5': No such file or directory


In [ ]:
# Create the dataset

im_transform=transforms.Compose([

transforms.ToTensor(),
transforms.Resize(train_resolution),

])





gta5_dataset = GTA5Dataset(root_dir='/content/Drive/MyDrive/GTA5', im_transform=im_transform)

print (len(gta5_dataset))


2500


In [ ]:
train_loader = DataLoader(gta5_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

**Citescapes Train Loader**

In [ ]:
# # Define a transform if you need to preprocess the images
# transformed_dataset = CityscapesDataset(root_dir='/content/Cityscapes/Cityspaces/gtFine/train', #/content/Cityscapes/Cityspaces/gtFine/train
# im_transform=transforms.Compose([

# transforms.ToTensor(),
# transforms.Resize(train_resolution),

# ]), )


# train_loader = DataLoader(transformed_dataset, batch_size=batch_size, shuffle=True, num_workers=4)




In [ ]:
# Load the DeepLabV2 model
#model = get_deeplab_v2(num_classes=19, pretrain=False)

# Load the BiSeNet model
model = BiSeNet(num_classes=19,context_path='resnet18') #'resnet18')  #'resnet101')  #BiSeNet


model = model.to(device)



# Define the CrossEntropyLoss with ignore_index set to 255
criterion = nn.CrossEntropyLoss(ignore_index=255)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 213MB/s]
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:01<00:00, 152MB/s]


In [ ]:
# Define the metric for mIoU
miou_metric = JaccardIndex(num_classes=19, task='multiclass' , ignore_index=255).to(device)


In [ ]:
# Function to compute latency
def measure_latency(model, input_tensor, repetitions=100):
    model.eval()
    with torch.no_grad():
        start = time.time()
        for _ in range(repetitions):
            _ = model(input_tensor)
        end = time.time()
    latency = (end - start) / repetitions
    return latency



# Measure FLOPs and number of parameters


dummy_input = torch.randn(1, 3, 1024, 512).to(device)

height = 1024
width = 512
image =torch.zeros((1,3, height, width)).to(device)   # torch.randn(1,3, 1024, 512).to(device)#

model.eval()
flops = FlopCountAnalysis(model, image)
print(flop_count_table(flops))



| module                                      | #parameters or shape   | #flops     |
|:--------------------------------------------|:-----------------------|:-----------|
| model                                       | 12.582M                | 25.78G     |
|  saptial_path                               |  0.371M                |  5.088G    |
|   saptial_path.convblock1                   |   1.856K               |   0.243G   |
|    saptial_path.convblock1.conv1            |    1.728K              |    0.226G  |
|    saptial_path.convblock1.bn               |    0.128K              |    16.777M |
|   saptial_path.convblock2                   |   73.984K              |   2.424G   |
|    saptial_path.convblock2.conv1            |    73.728K             |    2.416G  |
|    saptial_path.convblock2.bn               |    0.256K              |    8.389M  |
|   saptial_path.convblock3                   |   0.295M               |   2.42G    |
|    saptial_path.convblock3.conv1            |    0.2

In [ ]:
# Measure FLOPs and parameters using torchprofile
dummy_input = torch.randn(1, 3, 1024,512).to(device)
model.eval()
flops = torchprofile.profile_macs(model, args=(dummy_input,))

params = sum(p.numel() for p in model.parameters())

print(f' flops={flops}\n params={params} ')


 flops=25739266789
 params=12581672 


**IoU Calculation Function for Classes**

In [ ]:
def fast_hist(a, b, n):
    '''
    a and b are label and prediction respectively
    n is the number of classes
    '''
    #k = (a >= 0) & (a < n)
    k = (b >= 0) & (b < n)
    return np.bincount(n * a[k].astype(int) + b[k], minlength=n ** 2).reshape(n, n)


def per_class_iou(hist):
    epsilon = 1e-5
    return (np.diag(hist)) / (hist.sum(1) + hist.sum(0) - np.diag(hist) + epsilon)

**Training**

In [ ]:
#from pathlib import Path
#import os
#from google.colab import drive

# Mount Google Drive
#drive.mount('/content/drive')


from google.colab import drive
from pathlib import Path
import os

drive.mount('/content/Drive')
get_ipython().system('/content/Drive/MyDrive/Checkpoints/3a')
!ls

Drive already mounted at /content/Drive; to attempt to forcibly remount, call drive.mount("/content/Drive", force_remount=True).
/bin/bash: line 1: /content/Drive/MyDrive/Checkpoints/3a: Is a directory
datasets  models  README.md  train.py  utils.py


In [ ]:
# Function to save the model
def save_checkpoint(epoch, model, optimizer, save_dir='/content/Drive/MyDrive/Checkpoints/3a'):
    # Ensure the save directory exists
    Path(save_dir).mkdir(parents=True, exist_ok=True)

    # Define the model filename with the epoch number
    checkpoint_filename = f'checkpoint_epoch_{epoch}.pth'
    checkpoint_path = os.path.join(save_dir, checkpoint_filename)

    # Save the model and optimizer state dictionaries
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, checkpoint_path)
    print(f'Model and optimizer saved to {checkpoint_path}')

# Function to load the model
def load_checkpoint(epoch, model, optimizer, save_dir='/content/Drive/MyDrive/Checkpoints/3a'):
    checkpoint_filename = f'checkpoint_epoch_{epoch}.pth'
    checkpoint_path = os.path.join(save_dir, checkpoint_filename)

    # Check if the file exists
    if not os.path.isfile(checkpoint_path):
        raise FileNotFoundError(f"The specified file was not found: {checkpoint_path}")

    # Load the model and optimizer state dictionaries
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch'] + 1

    print(f'Model and optimizer loaded from {checkpoint_path}, resuming at epoch {start_epoch}')
    return model, optimizer, start_epoch



In [ ]:
# Training loop


# Resume training from the last checkpoint if available
resume_training =True   # Set this to True if you want to resume training

epochs = 50 # Set this to the total number of epochs you want to train
if resume_training:
    epoch_to_resume =20 #9,19,29,39,49  # Set this to the epoch from which you want to resume
    try:
        model, optimizer, start_epoch = load_checkpoint(epoch_to_resume, model, optimizer)

    except FileNotFoundError as e:
        print(e)
        start_epoch = 0
else:
    start_epoch = 0

# Training loop
for epoch in range(start_epoch, epochs):
#for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    miou_metric.reset()
    counter=1
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)

        #labels = labels.squeeze(1)

        loss = criterion(outputs[0], labels.long())
        #print(loss)
        loss.backward()
        optimizer.step()
        #print(counter)
        counter+=1

        running_loss += loss.item()

        miou_metric.update(outputs[0].argmax(dim=1), labels)



    # Save the model every 10 epochs
    if (epoch+1) % 3 == 0:
        save_checkpoint(epoch, model, optimizer)



    miou = miou_metric.compute().item()
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader)}, Train mIoU: {miou}')

# Measure latency after training
latency = measure_latency(model, dummy_input)
print(f"FLOPs: {flops}, Params: {params}, Latency: {latency:.6f} seconds")

print("Training completed!")


Model and optimizer loaded from /content/Drive/MyDrive/Checkpoints/3a/checkpoint_epoch_20.pth, resuming at epoch 21
Epoch [22/25], Loss: 0.09714148724079132, Train mIoU: 0.7728018164634705
Epoch [23/25], Loss: 0.08791104231476783, Train mIoU: 0.7934992909431458
Model and optimizer saved to /content/Drive/MyDrive/Checkpoints/3a/checkpoint_epoch_23.pth
Epoch [24/25], Loss: 0.08672812994718551, Train mIoU: 0.795935869216919
Epoch [25/25], Loss: 0.0841970064997673, Train mIoU: 0.7971428632736206
FLOPs: 25739266789, Params: 12581672, Latency: 0.015234 seconds
Training completed!


**Test Loader**

In [ ]:
# Define a transform if you need to preprocess the images
transformed_dataset = CityscapesDataset(root_dir='/content/Drive/MyDrive/Cityspaces/Cityspaces/gtFine/val', #/content/Cityscapes/Cityspaces/gtFine/train
im_transform=transforms.Compose([

transforms.ToTensor(),
transforms.Resize(test_resolution),

]), )


test_loader = DataLoader(transformed_dataset, batch_size=batch_size, shuffle=True, num_workers=4)




**Inference and mIoU calculation on test set**

In [ ]:


current_batch_size=labels.size(0)
model.eval()
miou_metric.reset()

all_classes_iou=np.zeros(19)
test_counter=0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)


        outputs = model(images)

        # Resize predictions to the original Cityscapes resolution
        outputs = TF.resize(outputs, test_resolution, interpolation=Image.NEAREST)
        print(outputs.shape , labels.shape)



        #calcualting IoU for each class and for each (image,label) pair separately

        for i in range (current_batch_size):

          mask=labels[i,:,:].cpu().numpy().flatten() != 255
          #hist=fast_hist(outputs[i,:,:,:].argmax(dim=0).cpu().numpy().flatten()[mask] , labels[i,:,:].cpu().numpy().flatten()[mask], 19)

          hist=fast_hist(outputs[i,:,:,:].argmax(dim=0).cpu().numpy().flatten() , labels[i,:,:].cpu().numpy().flatten(), 19)

          all_classes_iou=all_classes_iou+per_class_iou(hist)
          test_counter = test_counter +1









torch.Size([4, 19, 1024, 512]) torch.Size([4, 1024, 512])
torch.Size([4, 19, 1024, 512]) torch.Size([4, 1024, 512])
torch.Size([4, 19, 1024, 512]) torch.Size([4, 1024, 512])
torch.Size([4, 19, 1024, 512]) torch.Size([4, 1024, 512])
torch.Size([4, 19, 1024, 512]) torch.Size([4, 1024, 512])
torch.Size([4, 19, 1024, 512]) torch.Size([4, 1024, 512])
torch.Size([4, 19, 1024, 512]) torch.Size([4, 1024, 512])
torch.Size([4, 19, 1024, 512]) torch.Size([4, 1024, 512])
torch.Size([4, 19, 1024, 512]) torch.Size([4, 1024, 512])
torch.Size([4, 19, 1024, 512]) torch.Size([4, 1024, 512])
torch.Size([4, 19, 1024, 512]) torch.Size([4, 1024, 512])
torch.Size([4, 19, 1024, 512]) torch.Size([4, 1024, 512])
torch.Size([4, 19, 1024, 512]) torch.Size([4, 1024, 512])
torch.Size([4, 19, 1024, 512]) torch.Size([4, 1024, 512])
torch.Size([4, 19, 1024, 512]) torch.Size([4, 1024, 512])
torch.Size([4, 19, 1024, 512]) torch.Size([4, 1024, 512])
torch.Size([4, 19, 1024, 512]) torch.Size([4, 1024, 512])
torch.Size([4,

In [ ]:
test_counter

500

In [ ]:
all_classes_mIOU=(all_classes_iou/test_counter).round(3)  #calculating mean intersection over union for each class

In [ ]:
all_classes_mIOU

array([0.065, 0.08 , 0.   , 0.   , 0.001, 0.003, 0.   , 0.   , 0.005,
       0.   , 0.   , 0.016, 0.   , 0.   , 0.   , 0.001, 0.   , 0.001,
       0.   ])

In [ ]:
mIoU=all_classes_mIOU.mean()  #

In [ ]:
mIoU

0.009052631578947371

**GTA5 Labels**

In [ ]:
from abc import ABCMeta
from dataclasses import dataclass
from typing import Tuple


class BaseGTALabels(metaclass=ABCMeta):
    pass


@dataclass
class GTA5Label:
    ID: int
    color: Tuple[int, int, int]


class GTA5Labels_TaskCV2017(BaseGTALabels):
    road = GTA5Label(ID=0, color=(128, 64, 128))
    sidewalk = GTA5Label(ID=1, color=(244, 35, 232))
    building = GTA5Label(ID=2, color=(70, 70, 70))
    wall = GTA5Label(ID=3, color=(102, 102, 156))
    fence = GTA5Label(ID=4, color=(190, 153, 153))
    pole = GTA5Label(ID=5, color=(153, 153, 153))
    light = GTA5Label(ID=6, color=(250, 170, 30))
    sign = GTA5Label(ID=7, color=(220, 220, 0))
    vegetation = GTA5Label(ID=8, color=(107, 142, 35))
    terrain = GTA5Label(ID=9, color=(152, 251, 152))
    sky = GTA5Label(ID=10, color=(70, 130, 180))
    person = GTA5Label(ID=11, color=(220, 20, 60))
    rider = GTA5Label(ID=12, color=(255, 0, 0))
    car = GTA5Label(ID=13, color=(0, 0, 142))
    truck = GTA5Label(ID=14, color=(0, 0, 70))
    bus = GTA5Label(ID=15, color=(0, 60, 100))
    train = GTA5Label(ID=16, color=(0, 80, 100))
    motocycle = GTA5Label(ID=17, color=(0, 0, 230))
    bicycle = GTA5Label(ID=18, color=(119, 11, 32))

    list_ = [
        road,
        sidewalk,
        building,
        wall,
        fence,
        pole,
        light,
        sign,
        vegetation,
        terrain,
        sky,
        person,
        rider,
        car,
        truck,
        bus,
        train,
        motocycle,
        bicycle,
    ]

    @property
    def support_id_list(self):
        ret = [label.ID for label in self.list_]
        return ret

In [ ]:
class_labels = []
for label_name, label in GTA5Labels_TaskCV2017.__dict__.items():
  if isinstance(label, GTA5Label):
    class_labels.append(label_name)

In [ ]:
class_labels

['road',
 'sidewalk',
 'building',
 'wall',
 'fence',
 'pole',
 'light',
 'sign',
 'vegetation',
 'terrain',
 'sky',
 'person',
 'rider',
 'car',
 'truck',
 'bus',
 'train',
 'motocycle',
 'bicycle']

In [ ]:
import pandas as pd

In [ ]:
# Create a DataFrame
all_classes_mIOU_DF = pd.DataFrame({
'class_Label': class_labels,
'class_IoU': all_classes_iou
})

all_classes_mIOU_DF


,class_Label,class_IoU
0,road,32.431505
1,sidewalk,40.116434
2,building,0.000000
3,wall,0.000000
4,fence,0.389912
5,pole,1.641362
6,light,0.000000
7,sign,0.008262
8,vegetation,2.700807
9,terrain,0.000000
